# Hypothesis Testing

In [3]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plots
plots.style.use('fivethirtyeight')

import matplotlib.pylab as plt
%matplotlib inline

from scipy import special

from scipy import stats

## Ex 1:  Swain vs. Alabama ##

In [15]:
k = np.arange(101)
p = special.comb(100, k)*(0.26**k)*(0.74**(100-k))
#p = stats.binom.pmf(k, 100, .26),

fig, ax = plt.subplots()

ax.bar(k, p, width=1, ec='white');
ax.set_axisbelow(True)
ax.grid(alpha=0.25)
plt.xlim(1,60)
plt.title("Test Statistic Under Null Hypothesis: X~Bin(n=100, p=0.26)");
plt.xlabel("Number of Black People on Panel of 100")
plt.ylabel("Percent per unit")
plt.scatter(8, -.0001, color="red", s=30, label="Observed Test Statistic")
plt.legend()

In [ ]:
#Hypothesis Test:  
# Null:  p=0.26
# Alternative  p<0.26


# (1) Calculate the p-value: probability of region beyond observed value
print("P(X<=8) = ",p[0:9].sum())

# Or can do this directly using CDF:
from scipy import stats
stats.binom.cdf(8, 100, .26)
               


# (2) See if this is less than your pre-chosen significance level

**Conclusion:**

In [ ]:
#Conclusion:  since p<0.01, we reject the null hypothesis at the 1% significance level

## Ex 2: Alameda County Jury Panels ##

In [32]:
eth =['Asian', 'Black', 'Latino', 'White', 'Other']

jury = pd.DataFrame(
    {       
        'Eligible': [0.15, 0.18, 0.12, 0.54, 0.01],
         'Panels': [0.26, 0.08, 0.08, 0.54, 0.04]
        
    }, index = eth
)

jury

,Eligible,Panels
Asian,0.15,0.26
Black,0.18,0.08
Latino,0.12,0.08
White,0.54,0.54
Other,0.01,0.04


In [36]:
jury.plot.barh();


In [26]:
#Make an array with the eligible proportions assumed under the null hypothesis

null_hyp=np.array([0.15, 0.18, 0.12, 0.54, 0.01])



In [28]:
# Simulate a draw of size N assuming the null hypothesis:


N=1423  #from the ACLU report
simulated = stats.multinomial.rvs(N, null_hyp)

simulated_proportions = simulated/N

print(simulated)
print(simulated_proportions)

[228 264 166 753  12]
[0.16022488 0.18552354 0.11665495 0.52916374 0.00843289]


In [38]:
# The actual observed distribution (Panels) looks quite different
# from the simulation -- try running this several times to confirm!
jury_with_simulated=jury.copy()
jury_with_simulated["Simulated"]=simulated_proportions

print(jury_with_simulated)

jury_with_simulated.plot.barh()

        Eligible  Panels  Simulated
Asian       0.15    0.26   0.160225
Black       0.18    0.08   0.185524
Latino      0.12    0.08   0.116655
White       0.54    0.54   0.529164
Other       0.01    0.04   0.008433


<Axes: >

## Distance Between Distributions

In [ ]:

# In this case, we need to understand how each of the 5 categories
# differ from their expected values according to the model

jury_with_difference=jury.copy()
jury_with_difference["Difference"]=jury["Panels"]-jury["Eligible"]
jury_with_difference

## Total Variation Distance

In [ ]:
# This was the statistic we chose
def tvd(dist1, dist2):
    return sum(abs(dist1 - dist2))/2

In [ ]:
# The TVD of our observed data (Panels) from their expected values
# assuming the model is true 
obsvd_tvd = tvd(jury['Panels'],null_hyp)
obsvd_tvd

In [ ]:
# The TVD of a model simluation from its expected values
tvd(stats.multinomial.rvs(N, null_hyp)/N, null_hyp)

In [ ]:
def simulated_tvd():
    return tvd(stats.multinomial.rvs(N, null_hyp)/N, null_hyp)


num_simulations = 1000000

tvds = np.array([tvd(stats.multinomial.rvs(N, null_hyp)/N, null_hyp) for i in range(num_simulations)])


plt.hist(tvds, density=True);  
plt.xlabel("Total Distance Variation")
plt.ylabel("Percent per unit")
    

In [ ]:
# Add the observed test statistic to the plot:
plt.hist(tvds, density=True);  
plt.xlabel("Total Distance Variation")
plt.ylabel("Percent per unit")

    
plt.scatter(obsvd_tvd, -.001, color="red", s=40)

plt.show()



In [ ]:

# (1) Calculate the empirical p-value: simulation area beyond observed value

sum(tvds>=obsvd_tvd)/1000000

# (2) See if this is less than 1% (or our initially chosen significance level)

**Conclusion**:
Our empiricial p-value is $0$ (note that the actual theoretical p-value isn't 0, but since our simulations showed 0 observations out of $1000000$ that had a TVD greater than or equal to 0.14, we know that the theoretical p-value is at least less than $1/1000000$.

Given that our empirical p-value is less than our significance cutoff of $0.01$ we **REJECT THE NULL** hypothesis.      It is important for us to be precise about what we can conclude from our analysis:

**We can conclude that the distribution provided for the panelists who reported for service does not look like a random sample from the estimated distribution in the eligible population.**

Our analysis supports the ACLU’s conclusion that the panels were not representative of the distribution provided for the eligible jurors.

As with most such analyses, ours doesn’t say why the distributions are different.   See here for a more detailed discussion about this case:  https://inferentialthinking.com/chapters/11/2/Multiple_Categories.html

## Ex 3: Mendel and Pea Flowers ##

Mendel's null: 75% chance of purple flowers

Observed data:  grew 929 plants, of which 709 had purple flowers


In [ ]:
#Simulate one value of the test statistic under the null hypothesis:  75% chance of purple flowers out of sample of 929

predicted_proportions = np.array([.75, .25])

def test_statistic_purple_flowers(n=929):
    num_purple=stats.binom.rvs(n, 0.75)
    #print("Number of purple flowers out of 929:",num_purple)
    return abs(num_purple/n*100-75)

test_statistic_purple_flowers()

In [ ]:
# Simulate 10,000 copies of the test statistic
num_simulations = 10000;

outcomes=[]

for i in np.arange(num_simulations):
    outcomes = np.append(outcomes, test_statistic_purple_flowers(929))

plt.hist(outcomes, density=True);

#ax.set_axisbelow(True)
#ax.grid(alpha=0.25)
plt.xlabel("Abs(75-% Purple) in sample of 929 if the null true")
plt.ylabel("Percent per unit")
plt.show()


## Our Decision

In [ ]:
#Compare to observed test statistic:
## Mendel had 929 plants, of which 709 had purple flowers

observed_purples = 709 / 929*100

observed_test_statistic=abs(observed_purples-75)
observed_test_statistic


In [ ]:
#Plot

plt.hist(outcomes, density=True);
#ax.set_axisbelow(True)
#ax.grid(alpha=0.25)
plt.xlabel("Abs(75-% Purple) in sample of 929 if the null true")
plt.ylabel("Percent per unit")

plt.scatter(observed_test_statistic, -.0001, color="red", label = 'observed test statistic')
plt.legend()

plt.show()

In [ ]:
# (1) Calculate the EMPIRICAL p-value: simulation area beyond observed value

sum(outcomes>=observed_test_statistic)/ num_simulations

# (2) See if this is less than 5% (or our initially chosen significance level)


## Conclusion:  
Since 0.3655> 0.05, we **FAIL TO REJECT** the null hypothesis that the proportion of purple flowers is 0.75.  The evidence is consistent with our null hypothesis.  This does NOT prove that our null is true.  It just proves that we don't have evidence to reject it, so we will continue using the null hypothesis.


## Ex 4: The TA's Defense ##

In [ ]:
scores = pd.read_csv('scores_by_section.csv')
scores

In [ ]:
scores.groupby("Section").count()

In [ ]:
scores.groupby("Section").mean()

In [ ]:
observed_average = 13.6667 

In [ ]:
scores["Midterm"].mean()

In [ ]:
random_sample=scores.sample(27,replace=False)
random_sample["Midterm"].mean()

In [ ]:
random_sample['Midterm'].mean()

In [ ]:
# Simulate one value of the test statistic 
# under the hypothesis that the section is like a random sample from the class

def random_sample_midterm_avg():
    random_sample = scores.sample(27, replace=False)
    return random_sample['Midterm'].mean()

In [ ]:
# Simulate 100,000 copies of the test statistic


num_sim=100000

sample_averages = np.array([random_sample_midterm_avg() for i in range(num_sim)])




In [ ]:
# Compare the simulated distribution of the statistic
# and the actual observed statistic

plt.hist(sample_averages, density=True);

plots.scatter(observed_average, -0.01, color='red', s=120);

## Calculating p-value

In [ ]:
# (1) Calculate the p-value: simulation area beyond observed value
np.count_nonzero(sample_averages <= observed_average) / num_sim
# (2) See if this is less than 5%

## Conclusion:  
Since 0.05746> 0.05, we **FAIL TO REJECT** the null hypothesis that any difference in scores for section 3 was just due to random chance

In [ ]:
from scipy import stats


## Theoretical Calculation:

We could have done this without using simulations thanks to the Central Limit Theorem!   

By the CLT, $$\bar{X}_{27} \sim N(\mu, \sigma^2/27)$$ where $\mu = $ pop mean = `scores["Midterm"].mean()`   and                    

$\sigma = $ pop_std = `pop_std=scores["Midterm"].std(ddof=0)`


Thus 

p-value =   $P(\bar{X}_{27}<=13.667)$ = `stats.norm.cdf(13.667,pop_mean, pop_std/np.sqrt(27))`


In [ ]:
pop_mean=scores["Midterm"].mean()

In [ ]:
pop_std=scores["Midterm"].std(ddof=0)

In [ ]:
stats.norm.cdf(13.667,pop_mean, pop_std/np.sqrt(27))

Again, since $p= 0.059> 0.05$, we **FAIL TO REJECT** the null hypothesis that any difference in scores for section 3 was just due to random chance